In [1]:
import gzip
import pandas as pd
with gzip.open("goodreads_books.json.gz", 'r') as f:
    line = f.readline()

In [2]:
import json
json.loads(line)

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

In [ ]:
with gzip.open("goodreads_books_authors.json.gz", 'r') as f:
    line = f.readline()

In [9]:
def parse_books(line):
    book = json.loads(line)
    return {
        "book_id": book["book_id"],
        "title": book["title_without_series"],
        "ratings": book["ratings_count"],
        "url": book["url"],
        "cover": book["image_url"],
    }

In [10]:
# Narrow down search by only including books for which there are at least 20 ratings. (Books with less than 20 ratings are obviously not very popular and thus are not necessary inclusions in our search engine).
# Append all of the books that match these requirements into a list called title.
titles = []
with gzip.open("goodreads_books.json.gz", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        fields = parse_books(line)
        try:
            ratings = int(fields["ratings"])
        except ValueError:
            continue
        if ratings > 100:
            titles.append(fields)

In [12]:
import pandas as pd
import csv
with open('book_fields.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["book_id", "title", "ratings", "url", "cover"])
    writer.writeheader()
    writer.writerows(titles)

In [10]:
book_titles = pd.read_csv('book_fields.csv')

In [13]:
all_interactions = []
count = 0
all_interactions.append(['user_id', 'book_id', 'is_read', 'rating', 'is_reviewed'])
with open("goodreads_interactions.csv", "r") as f:
        while True:
            if count > 5000000:
                break
            line = f.readline()
            if not line:
                break
            user_id, csv_id, _, rating, _ = line.split(",")
            
            try:
                rating = int(rating)
            except ValueError:
                continue
            if count < 5000000:
                all_interactions.append(line.strip().split(","))
            count+=1

In [14]:
import csv
with open('all_interactions.csv', 'w', encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_NONE)
        #writer.writeheader()
        writer.writerows(all_interactions)